In [45]:
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from torch import optim
import numpy as np

batch_size = 10

In [46]:
# Generate dummy data
input_data = torch.rand((100, 30, 40))
output_data = torch.rand((100, 20, 10))

In [47]:
# Data loader
class Loader(Dataset):
    
    def __len__(self):
        return input_data.size()[0]
    
    def __getitem__(self, idx):  # idx is index for subset of batch data.
        input_frame = input_data[idx]
        output_words = output_data[idx]
        return input_frame, output_words

In [52]:
class EncoderLSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = 1
        self.Lstm = nn.LSTM(input_size = self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, bidirectional=True)
    
    def forward(self, input_state, hidden_state):
        output, hidden = self.Lstm(input_state, hidden_state)
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(2,batch_size,self.hidden_size), torch.zeros(2,batch_size,self.hidden_size))
        
class DecoderLSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = 1
        self.Lstm = nn.LSTM(input_size = self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, bidirectional=True)
    
    def forward(self, input_state, hidden_state):
#         print('forward hidden_state', hidden_state.size())
        output, hidden = self.Lstm(input_state, hidden_state)
        return output, hidden
    

In [55]:
loader = Loader()
dataloader = DataLoader(loader, batch_size=batch_size,shuffle=True)

enc_input_size = 40
dec_input_size = 10

hidden_size = 128


print('hello')

for idx, sample in enumerate(dataloader):
    print(idx)
    a, b = sample[0], sample[1]
    print(a.size(), b.size())
    encoder = EncoderLSTM(enc_input_size, hidden_size)
    encoder_hidden = encoder.initHidden()
    encoder_output, encoder_hidden = encoder(a, encoder_hidden)
    
    decoder_hn = torch.unsqueeze(encoder_output[:,0,:], 1).transpose(0,1).view(2, batch_size, -1)
    decoder_hidden = (decoder_hn, decoder_hn)
    decoder = DecoderLSTM(dec_input_size, hidden_size)
    decoder_output, decoder_hidden = decoder(b, decoder_hidden)
    
    print('encoder hidden', type(encoder_hidden))
    print('output', encoder_output.size())
    print('decoder output', type(decoder_output))
    print('decoder hidden', type(decoder_hidden))
    break
    
   

hello
0
torch.Size([10, 30, 40]) torch.Size([10, 20, 10])
encoder hidden <class 'tuple'>
output torch.Size([10, 30, 256])
decoder output <class 'torch.Tensor'>
decoder hidden <class 'tuple'>


In [ ]:
a = torch.zeros((10,20,30))
b = a[:,0,:]
print(b.size())
b = torch.unsqueeze(b, 1)
print(b.size())